# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [1]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [2]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

/var/folders/j4/3r_t681d30n9lfr4srl0_2ym0000gq/T/ipykernel_74229/967410264.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
/var/folders/j4/3r_t681d30n9lfr4srl0_2ym0000gq/T/ipykernel_74229/967410264.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)


,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [4]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

/var/folders/j4/3r_t681d30n9lfr4srl0_2ym0000gq/T/ipykernel_74229/3705894443.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']


Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [5]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
date,,
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [6]:
df.head()

,fng_value,Close
date,,
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [7]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [8]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 0
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [9]:
# Use 70% of the data for training and the remaineder for testing
from sklearn.model_selection import train_test_split


# Split the data into training and test sets with a 70%/30% ratio
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (372, 10)
X_test shape: (160, 10)
y_train shape: (372, 1)
y_test shape: (160, 1)


In [10]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Fit the scaler on the training data and transform both training and test data
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [11]:
# Reshape the features for the model
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

print(X_train.shape)

(372, 10)


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

time_steps = 10
num_features = 1

# Initialize the model
model = Sequential()

# Add first LSTM layer
model.add(LSTM(units=64, return_sequences=True, input_shape=(time_steps, num_features)))
# Add first Dropout layer
model.add(Dropout(0.2))

# Add second LSTM layer
model.add(LSTM(units=32, return_sequences=True))
# Add second Dropout layer
model.add(Dropout(0.2))

# Add third LSTM layer
model.add(LSTM(units=16))
# Add third Dropout layer
model.add(Dropout(0.2))

# Add final Dense layer for regression
model.add(Dense(units=1, activation='linear'))


In [14]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [15]:
# Summarize the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 64)            16896     
                                                                 
 dropout (Dropout)           (None, 10, 64)            0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 32)            12416     
                                                                 
 dropout_1 (Dropout)         (None, 10, 32)            0         
                                                                 
 lstm_2 (LSTM)               (None, 16)                3136      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 1

In [16]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended

model.fit(X_train_reshaped, y_train_scaled, epochs=100, batch_size=15, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100


2024-06-15 20:58:20.456166: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


20/20 [==============================] - 2s 25ms/step - loss: 0.0870 - val_loss: 0.0393
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0536 - val_loss: 0.0397
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0467 - val_loss: 0.0392
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0471 - val_loss: 0.0408
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0522 - val_loss: 0.0396
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0511 - val_loss: 0.0395
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0484 - val_loss: 0.0407
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.0477 - val_loss: 0.0423
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0495 - val_loss: 0.0396
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.0471 - val_loss: 0.0401
Epoch 11/100
20/20 [

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [17]:
# Evaluate the model
loss = model.evaluate(X_test_reshaped, y_test_scaled)
print(f"Test Loss: {loss}")


5/5 [==============================] - 0s 2ms/step - loss: 0.0468
Test Loss: 0.04676210880279541


In [18]:
# Make some predictions
predictions_scaled = model.predict(X_test_reshaped)

from sklearn.metrics import r2_score
r_squared = r2_score(y_test_scaled, predictions_scaled)
print(f"R-squared: {r_squared}")

5/5 [==============================] - 0s 2ms/step
R-squared: 0.20508107217161053


In [19]:
# Recover the original prices instead of the scaled version
predictions = scaler_y.inverse_transform(predictions_scaled)
real_prices = scaler_y.inverse_transform(y_test_scaled.reshape(-1, 1))

In [20]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predictions.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
date,,
2019-02-20,6375.080078,6318.390625
2019-02-21,8533.000000,6321.860352
2019-02-22,8550.669922,7989.828613
2019-02-23,12355.059570,11692.218750
2019-02-24,6488.720215,5943.019043


In [21]:
# Plot the real vs predicted values as a line chart
stocks.hvplot.line(x='date', y=['Real', 'Predicted'], title="Real vs Predicted Values", ylabel="Price", xlabel="Date")

:NdOverlay   [Variable]
   :Curve   [date]   (value)